In [1]:
# ================================================================
# 📦 KOD BLOĞU 1: Ortam Kurulumu ve Model Tanımları (NİHAİ VERSİYON)
# ================================================================

import os
import re
import warnings
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Set 

from dotenv import load_dotenv, find_dotenv

# Hugging Face dosya okuması için gerekli (Kod Bloğu 2'de kullanılacak)
import pandas as pd 
import fsspec # Pandas'ın hf:// uzantısını okuması için gerekli

# LangChain ve LLM İthalatları
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import numpy as np

# Hugging Face'den veri okumak için gerekli
from huggingface_hub import HfFileSystem # fsspec ile birlikte çalışır

warnings.filterwarnings("ignore")

print("✅ Kütüphaneler yüklendi")

# API Key ve LLM Setup
api_key = None
try:
    load_dotenv(find_dotenv(usecwd=True, raise_error_if_not_found=True))
    api_key = os.getenv('GOOGLE_API_KEY')
    
    # Not: genai.configure() satırı, önceki Pydantic/LangChain çakışmalarını önlemek için kaldırılmıştır.
    
    if api_key:
        print("✅ API Anahtarı yüklendi")
    else:
        print("⚠️ .env dosyası bulunamadı/yüklenemedi veya API anahtarı boş. Lütfen ortam değişkenlerini kontrol edin.")
        # LLM tanımlama hatası vermemesi için boş bir değer atandı (LLM bu durumda hata verecektir).
except Exception:
    print("⚠️ .env dosyası bulunamadı/yüklenemedi. Lütfen API anahtarının ortam değişkenlerinde olduğundan emin olun.")

# LLM Tanımları
try:
    # Versiyon hataları giderildiği için LangChain sarmalayıcısı kullanılıyor.
    LLM = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.0,
        max_output_tokens=1024,
        google_api_key=api_key # API anahtarı LangChain'e doğrudan iletiliyor
    )

    LLM_PREDICTOR = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.3, 
        max_output_tokens=512,
        google_api_key=api_key # API anahtarı LangChain'e doğrudan iletiliyor
    )

    print(f"✅ Ana LLM Modeli hazır: {LLM.model}")

except Exception as e:
    print(f"❌ LLM Tanımlama Hatası: {e}")
    LLM = None
    LLM_PREDICTOR = None
    print("⚠️ LLM tanımlanamadığı için RAG zinciri çalışmayacaktır.")


# ================================================================
# ⚙️ HUGGING FACE VERİ SETİ AYARLARI (Kod Bloğu 2 için gerekli)
# ================================================================
# NOT: Bu değişkenler Kod Bloğu 2'deki Pandas/fsspec okumasında kullanılmayacaktır, 
# ancak kod temizliği için burada tutulabilir.
DATASET_NAME = "Renicames/turkish-law-chatbot" 
SPLIT_NAME = "train" 

print(f"✅ Veri Seti Ayarları Hazır: {DATASET_NAME} / {SPLIT_NAME}")


# Embedding Model ve ChromaDB Ayarları 
CHROMA_PATH = "./chroma_db_lexmove_mini" # Notebook'un çalıştığı dizin içine yazar.

COLLECTION_NAME = "mevzuat_chunks_mini" 
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

# Embedding Model
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# ChromaDB Bağlantısı
vectorstore: Optional[Chroma] = None 

try:
    vectorstore = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings,
        collection_name=COLLECTION_NAME
    )
    print(f"✅ ChromaDB yüklendi: {COLLECTION_NAME}")
    # print(f"📊 Toplam chunk sayısı: {vectorstore._collection.count()}") # Chroma 0.5.x'te _collection hatası verebilir
    
    # ------------------------------------------------
    # YENİ EKLENTİ: Kanonik Kanun Adını Çekme (Çalışan fonksiyon korundu)
    # ------------------------------------------------
    def get_canonical_law_list(db: Chroma) -> Set[str]:
        if db is None:
            return set()
        
        all_metadata = db._collection.get(limit=1000)['metadatas']
        
        law_set = {
            doc.get('kanun_adi', 'Bilinmeyen Kanun')
            for doc in all_metadata
            if doc.get('kanun_adi')
        }
        return law_set
    
    CANONICAL_LAW_LIST = get_canonical_law_list(vectorstore)
    print(f"📚 Kanonik Kanun Listesi Yüklendi: {CANONICAL_LAW_LIST}")
    
except Exception as e:
    print(f"❌ ChromaDB bağlantı hatası: {e}. Lütfen yolu kontrol edin.")
    print("⚠️ RAG sisteminin çalışması için ChromaDB gereklidir.")
    vectorstore = None
    CANONICAL_LAW_LIST = set()

/Users/asli/Desktop/Github-Projeleri/LexMove-ChatBot/venv_final/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Kütüphaneler yüklendi
✅ API Anahtarı yüklendi
✅ Ana LLM Modeli hazır: models/gemini-2.0-flash
✅ Veri Seti Ayarları Hazır: Renicames/turkish-law-chatbot / train


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


✅ ChromaDB yüklendi: mevzuat_chunks_mini
📚 Kanonik Kanun Listesi Yüklendi: set()


In [ ]:
# ================================================================
# 🚨 KRİTİK HATA DÜZELTME (PATCH) - pydantic_v1 hatası için
# ================================================================

import sys
from pydantic import v1

try:
    # Python'a, 'langchain_core.pydantic_v1' çağrıldığında
    # onu 'pydantic.v1' olarak kabul etmesini söylüyoruz.
    sys.modules['langchain_core.pydantic_v1'] = v1
    print("✅ Pydantic V1/V2 uyumsuzluk hatası için geçici düzeltme (patch) uygulandı.")
except Exception as e:
    print(f"❌ Patch uygulamada hata: {e}")
    
# ================================================================

In [2]:
# ================================================================
# 📦 KOD BLOĞU 2: Pandas ile Veri Yükleme ve Document Oluşturma (FİNAL)
# ================================================================

# GERÇEK SÜTUN ADLARI (Veri setindeki sütun isimleri)
SORU_COL = 'Soru'  
CEVAP_COL = 'Cevap'     

def load_and_create_documents() -> List[Document]:
    """
    Pandas'ın özel 'hf://' yolu ile Hugging Face Q&A veri setini yükler 
    ve LangChain Document listesine dönüştürür.
    """
    all_documents = []
    
    # Hugging Face Veri Seti Yolu
    try:
        splits = {'train': 'train.json', 'test': 'test.json'}
        # Doğru Hugging Face yol formatı (Kod Bloğu 1'deki DATASET_NAME ile birleştirilmedi)
        dataset_path = "hf://datasets/Renicames/turkish-law-chatbot/" + splits["train"]
        
        print(f"⏳ Hugging Face'ten veri seti yükleniyor (Pandas/fsspec): {dataset_path}")
        
        # Pandas ile JSON dosyasını doğrudan Hugging Face'ten okuma
        df = pd.read_json(dataset_path)
        
    except Exception as e:
        # Fsspec veya Pandas yükleme hatası
        print(f"❌ Veri seti yüklenirken HATA oluştu: {e}")
        return []
        
    print(f"✅ Veri seti yüklendi. Toplam satır: {len(df)}")
    
    # DataFrame satırlarını LangChain Document'a dönüştürme
    for index, row in df.iterrows():
        try:
            question = str(row[SORU_COL]).strip()
            answer = str(row[CEVAP_COL]).strip()
            
            page_content = answer 
            
            # Soru ve Cevap çok kısa ise atla
            if len(page_content) < 30:
                continue
                
            metadata = {
                "source": dataset_path, # Veri yolunu kaynak olarak sakla     
                "question": question,       
                "kanun_adi": "ÇEŞİTLİ MEVZUAT",
                "madde_no": f"Satır {index}"
            }
            
            document = Document(
                page_content=page_content,
                metadata=metadata
            )
            all_documents.append(document)
            
        except KeyError:
            print(f"❌ Kritik Hata: Sütun adları ('{SORU_COL}' veya '{CEVAP_COL}') yanlış.")
            return []
        except Exception as e:
            print(f"❌ Satır {index} işlenirken hata: {e}")
            continue
            
    print(f"\n📚 TOPLAM {len(all_documents)} Document nesnesi hazırlandı.")
    return all_documents

# Tüm veriyi yükle ve Document listesini oluştur
documents = load_and_create_documents()

⏳ Hugging Face'ten veri seti yükleniyor (Pandas/fsspec): hf://datasets/Renicames/turkish-law-chatbot/train.json
✅ Veri seti yüklendi. Toplam satır: 13354

📚 TOPLAM 13138 Document nesnesi hazırlandı.


In [3]:
# ================================================================
# 📦 KOD BLOĞU 3: ChromaDB Oluşturma/Güncelleme (Kilit Sorununa Karşı Düzeltme)
# ================================================================

# Gerekli import'lar (Kod Bloğu 1'den 'from pathlib import Path' ve 'shutil' importlarını kontrol edin)
import shutil 
from pathlib import Path 
from typing import List
from langchain_core.documents import Document

# 🚨 KRİTİK İTHALAT: ChromaDB'nin ayarlarını değiştirmek için
from chromadb.config import Settings
# ... (Diğer değişkenler Code Block 1 ve 2'den geliyor)

def create_or_update_chromadb(docs: List[Document]):
    """Yeni verileri ChromaDB'ye yazar ve eski veriyi siler."""
    
    # 1. Eski ChromaDB'yi silme (Temiz bir başlangıç için)
    if Path(CHROMA_PATH).exists():
        print(f"⚠️ Eski veritabanı siliniyor: {CHROMA_PATH}")
        try:
            shutil.rmtree(CHROMA_PATH)
        except OSError as e:
            print(f"❌ Klasör silinirken hata: {e}")
            return None
    
    if not docs:
        print("❌ Veritabanına yazılacak doküman yok. İşlem iptal edildi.")
        return None

    print(f"⏳ Veritabanı oluşturuluyor ve {len(docs)} parça yazılıyor...")
    
    # 2. ChromaDB'ye yazma (LOCKING'i devre dışı bırakıyoruz)
    try:
        # 🚨 DÜZELTME: Settings ile kilit mekanizmasını değiştirme
        db_settings = Settings(
            # Kilit mekanizmasını basitleştirerek erişim hatasını aşmayı deniyoruz
            anonymized_telemetry=False, 
            allow_reset=True # Reset izni vererek esnekliği artır
        )

        db = Chroma.from_documents(
            docs, 
            embeddings, 
            persist_directory=CHROMA_PATH, 
            collection_name=COLLECTION_NAME,
            client_settings=db_settings # Yeni ayarları uygulayın
        )
    except Exception as e:
        print(f"[HATA] ChromaDB oluşturulurken kritik sorun oluştu: {e}")
        return None
    
    # 3. Kalıcı hale getirme ve sonuç
    db.persist()
    print("✅ ChromaDB başarıyla oluşturuldu/güncellendi.")
    print(f"📊 Toplam parça sayısı: {db._collection.count()}")
    return db

# ChromaDB'yi oluştur
if 'documents' in locals() and documents is not None:
    mini_vectorstore = create_or_update_chromadb(documents)
else:
    print("⚠️ Hata: 'documents' değişkeni (Kod Blok 2'den) tanımlı değil veya boş.")
    mini_vectorstore = None

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


⚠️ Eski veritabanı siliniyor: ./chroma_db_lexmove_mini
⏳ Veritabanı oluşturuluyor ve 13138 parça yazılıyor...
✅ ChromaDB başarıyla oluşturuldu/güncellendi.
📊 Toplam parça sayısı: 13138


In [4]:
# ================================================================
# 📦 KOD BLOĞU 4 (TEK BLOK): RAG Zinciri Kurulumu ve Tanımları
# ================================================================

# Not: LLM, mini_vectorstore (db) ve gerekli import'ların 
# (Document, List, ChatPromptTemplate, StrOutputParser) önceki bloklarda tanımlanmış olduğunu varsayıyoruz.

from typing import List
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma # Chroma'yı import etmeliyiz.

# --- RAG Yardımcı Fonksiyonları ---

def format_docs(docs: List[Document]) -> str:
    """Çekilen document listesini tek bir string bağlamına dönüştürür."""
    # Q&A içeriğinde yalnızca page_content'ı birleştiriyoruz.
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

def retrieve_relevant_chunks(query: str, db: Chroma, k: int = 10) -> List[Document]:
    """Sorguya göre ChromaDB'den en alakalı parçaları çeker."""
    # mini_vectorstore'un global olarak tanımlandığını varsayıyoruz (Kod Bloğu 3'ten).
    if db is None:
        return []
    # k=10, daha fazla bağlam çekerek cevabın kalitesini artırır
    return db.similarity_search(query=query, k=k)


# --- Prompt Tanımı ---

RAG_PROMPT_TEMPLATE = """
Sen, Türk hukuku konusunda uzmanlaşmış, yapay zeka destekli bir danışmansın.
Görevinde sadece, sana aşağıda sağlanan 'BAĞLAM' içerisindeki bilgileri kullanmalısın.
Eğer BAĞLAM'daki bilgiler soruyu yanıtlamak için yeterli değilse, kesinlikle uydurma yapma veya genel bilgi verme.
Böyle bir durumda cevabın "Üzgünüm, bu soruya yanıt verebilecek yeterli ve doğrulanmış hukuki bilgiye BAĞLAM'da ulaşamadım." olmalıdır.

Aşağıdaki kurallara kesinlikle uy:
1. Cevaplarını Türkçe ve kibar bir dille ver.
2. Cevabını BAĞLAM'daki bilgilere dayandırarak oluştur.
3. Yanıtın, çekilen BAĞLAM'daki en detaylı ve açıklayıcı bilgi parçasını içermeli ve bu bilgiyi olabildiğince eksiksiz sunmalıdır.
4. Çekilen BAĞLAM'daki hukuki açıklamanın tamamını kullanmaya çalış.

BAĞLAM:
{context}

KULLANICININ SORUSU:
{question}

SENİN YANITIN:
"""
RAG_PROMPT = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)


# --- RAG Zinciri Oluşturma ---

if 'mini_vectorstore' in locals() and mini_vectorstore is not None:
    # Lambda fonksiyonunda mini_vectorstore nesnesini kullanıyoruz.
    rag_chain = (
        {"context": lambda x: format_docs(retrieve_relevant_chunks(x["question"], mini_vectorstore)),
         "question": lambda x: x["question"]}
        | RAG_PROMPT  
        | LLM  
        | StrOutputParser()
    )
    print("✅ RAG Zinciri (rag_chain) başarıyla oluşturuldu.")
else:
    print("❌ Hata: ChromaDB (mini_vectorstore) tanımlı olmadığı için RAG zinciri oluşturulamadı.")
    rag_chain = None # Hata oluşursa rag_chain'i None olarak tanımla

✅ RAG Zinciri (rag_chain) başarıyla oluşturuldu.


In [5]:
# ================================================================
# 📦 KOD BLOĞU SON: İnteraktif RAG Chatbot Test Arayüzü
# ================================================================

# Bu blok, terminal tabanlı basit bir sohbet döngüsü oluşturur.
# Notebook'ta LLM ve RAG zincirinin düzgün çalıştığını test etmek için idealdir.

print("="*60)
print("📚 İNTERAKTİF RAG CHATBOT BAŞLATILDI (Çıkmak için 'exit' yazın)")
print("="*60)

# Sohbet döngüsü
while True:
    # Kullanıcıdan giriş al
    user_input = input("\n👤 Soru: ")
    
    # Çıkış komutunu kontrol et
    if user_input.lower() in ["exit", "quit", "çık"]:
        print("\nChatbot kapatılıyor. Hoşça kalın!")
        break
    
    if not user_input.strip():
        continue
        
    try:
        print("🤖 LexMove: Cevap hazırlanıyor... (Lütfen bekleyin)")
        
        # 🚨 Notebook'taki RAG Zincirini Çalıştırma
        # rag_chain'in önceki bloklarda tanımlanmış olduğunu varsayıyoruz.
        
        response = rag_chain.invoke({"question": user_input})
        
        # Yanıtı ekrana bas
        print("\n💬 Cevap:")
        print("-" * 5)
        print(response)
        print("-" * 5)
        
    except Exception as e:
        # API veya zincir hatası durumunda (Örn: API limit)
        error_type = e.__class__.__name__
        print(f"\n❌ Hata: RAG Zinciri Çalıştırılamadı. Detay: {error_type}")
        print("Lütfen API anahtarınızı veya limitinizi kontrol edin.")

# Döngü bitti.
print("\n" + "="*60)

📚 İNTERAKTİF RAG CHATBOT BAŞLATILDI (Çıkmak için 'exit' yazın)



👤 Soru:  cumhurbaşkanı görev süresi


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


🤖 LexMove: Cevap hazırlanıyor... (Lütfen bekleyin)

💬 Cevap:
-----
Cumhurbaşkanı'nın görev süresi beş yıldır. Anayasa'ya göre, bir kişi en fazla iki defa cumhurbaşkanı seçilebilir. Cumhurbaşkanı, görev süresi dolmadan istifa ederek görevden çekilebilir. Görev süresi beş yılın sonunda sona eren cumhurbaşkanı, yeniden aday olup seçilmezse görevini yeni seçilen cumhurbaşkanına devreder.
-----



👤 Soru:  kamu ihalesi


🤖 LexMove: Cevap hazırlanıyor... (Lütfen bekleyin)

💬 Cevap:
-----
Kamu ihale hukuku, kamu kurumlarının mal veya hizmet alımı ile yapım işlerini düzenleyen hukuk dalıdır. Kamu İhale Kanunu kapsamına kamu kurumlarının mal ve hizmet alımları, yapım işleri, kiralama ve danışmanlık hizmetleri gibi çeşitli işler girer. Kamu İhale Kanunu'na göre ihaleler, açık ihale usulü, belli istekliler arasında ihale usulü, pazarlık usulü ve doğrudan temin gibi çeşitli yöntemlerle yapılabilir. Kamu İhale Kanunu'nun şeffaflığı, ihale ilanlarının ve sonuçlarının kamuoyuna açık bir şekilde duyurulması, ihale dokümanlarının erişilebilir olması ve ihale süreçlerinin izlenebilirliği ile sağlanır. Kamu İhale Kanunu'na aykırı davranışlar için çeşitli idari ve cezai yaptırımlar uygulanabilir. Bu yaptırımlar arasında ihale yasakları, para cezaları ve ceza davaları bulunmaktadır. Kamu İhale Kanunu'na göre ihalelere ilişkin itiraz ve şikayetler, Kamu İhale Kurumu'na yapılır. Kurum, başvuruları değerlendirir ve gerek

KeyboardInterrupt: Interrupted by user

In [ ]:
# ================================================================
# 📦 KOD BLOĞU TEST: API Anahtarının Doğrudan Kontrolü
# ================================================================

import google.generativeai as genai
import os 
from dotenv import load_dotenv, find_dotenv

# API Anahtarını tekrar yükle
load_dotenv(find_dotenv(usecwd=True, raise_error_if_not_found=False))
test_api_key = os.getenv('GOOGLE_API_KEY')

if test_api_key:
    try:
        # genai yapılandırması (Bu, LangChain'den farklıdır ve doğrudan API testidir)
        genai.configure(api_key=test_api_key)
        
        # Basit bir model çağrısı yap
        model = genai.GenerativeModel('gemini-2.5-flash')
        
        print("⏳ Basit bir API çağrısı yapılıyor...")
        response = model.generate_content("Merhaba de.")
        
        print("✅ API BAĞLANTISI BAŞARILI!")
        print(f"Alınan Yanıt: {response.text.strip()}")
        
    except Exception as e:
        print(f"❌ API BAĞLANTISI BAĞIMSIZ TESTTE BAŞARISIZ OLDU!")
        print(f"Hata Detayı: {e.__class__.__name__} - {e}")
        print("Bu, API anahtarınızın geçersiz olduğu veya kotanın gerçekten tükendiği anlamına gelir.")
else:
    print("❌ API Anahtarı hala okunamıyor.")

In [ ]:
import os

# Notebook'ta kullandığın yol neydi?
CHROMA_PATH = os.path.expanduser("~/chroma_db_lexmove_mini")
print(f"Notebook yolu: {CHROMA_PATH}")
print(f"Var mı? {os.path.exists(CHROMA_PATH)}")

# Proje içindeki yol
PROJECT_PATH = "/Users/asli/Desktop/Github-Projeleri/LexMove-ChatBot/chroma_db_lexmove_mini"
print(f"\nProje yolu: {PROJECT_PATH}")
print(f"Var mı? {os.path.exists(PROJECT_PATH)}")

# Home directory'de var mı?
HOME_PATH = os.path.expanduser("~/chroma_db_lexmove_mini")
print(f"\nHome yolu: {HOME_PATH}")
print(f"Var mı? {os.path.exists(HOME_PATH)}")